# Survival analysis

In [ ]:
import pickle
num_cores = 4
import multiprocessing as mp
import time
import random
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from scipy.stats import ks_2samp, mannwhitneyu,ttest_ind, spearmanr
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm

from datetime import date

import seaborn as sns
sns.set_palette("colorblind")

from sklearn.utils import resample

from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import roc_curve,precision_recall_curve
from sklearn.feature_selection import chi2,f_regression

dir_="../../data/"

In [ ]:
X_all_proteins = pd.read_csv(dir_+'integrated_X_raw_all_proteins.csv',index_col=0)
X_all_clinical = pd.read_csv(dir_+'integrated_X_clinical_and_cohort_covariates.csv',index_col=0)
Y_pgd = pd.read_csv(dir_+'integrated_pgd_y.csv',index_col=0,header=None)
Y_pgd = Y_pgd.rename(columns={1 : 'PGD'})
Y_pgd.index.name=''

In [ ]:
cov_df = X_all_clinical.loc[:,['Cohort_Columbia','Cohort_Cedar']].copy().astype(int)
all_cov_df = cov_df.copy()
all_cov_df.loc[:,'Cohort_Paris'] = (
    (all_cov_df['Cohort_Columbia'] + 
     all_cov_df['Cohort_Cedar'])==0).astype(int)

In [ ]:
idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()
idmap_sub

In [ ]:
proteins = pickle.load(open(dir_+'proteins_no_immunoglobulins.pkl','rb'))
ig_proteins = pickle.load(open(dir_+'proteins_immunoglobulins.pkl','rb'))
print(len(proteins))
print(len(ig_proteins))

## Raw mortality data input and reformatted output

In [ ]:
Y_mortality = \
pd.concat(
    [
        (pd.read_csv(dir_+'mortality_cumc.csv',dtype={'subject' : str}).set_index('subject').
         loc[:,['expired','graft_date','expiration_date']]),
        (pd.read_csv(dir_+'mortality_cedars.csv',dtype={'subject' : str}).set_index('subject').
         loc[:,['expired','graft_date','expiration_date']]),
        (pd.read_csv(dir_+'mortality_paris.csv',dtype={'subject' : str}).set_index('subject').
         loc[:,['expired','graft_date','expiration_date']])
    ]
)
Y_mortality.graft_date = pd.to_datetime(Y_mortality.graft_date,format='%m/%d/%y')
Y_mortality.expiration_date = pd.to_datetime(Y_mortality.expiration_date,format='%m/%d/%y')
Y_mortality['deathdays'] = (Y_mortality.expiration_date - Y_mortality.graft_date)
Y_mortality['days_to_death'] = (Y_mortality.expiration_date - Y_mortality.graft_date).dt.days

In [ ]:
X_all_clinical.loc[:,'MELDXI'] = \
(3.78 * np.log(X_all_clinical.loc[:,'TBILI'])) + \
(9.57 * np.log(X_all_clinical.loc[:,'Creatinine'])) + 6.43

In [ ]:
joined = \
(
    X_all_clinical.
    join(Y_mortality).
    join(Y_pgd)
)

In [ ]:
joined['Mechanical_Support_Type'] = np.nan
joined.loc['53190','Mechanical_Support_Type'] = 'HM2'
joined.loc['53488','Mechanical_Support_Type'] = 'HM2'
joined.loc['54314','Mechanical_Support_Type'] = 'HM2'
joined.loc['55175','Mechanical_Support_Type'] = 'HM2'
joined.loc['56118','Mechanical_Support_Type'] = 'HW'
joined.loc['56191','Mechanical_Support_Type'] = 'HM2'
joined.loc['56767','Mechanical_Support_Type'] = 'HM2'
joined.loc['57267','Mechanical_Support_Type'] = 'HW'
joined.loc['57268','Mechanical_Support_Type'] = 'HW'
joined.loc['57270','Mechanical_Support_Type'] = 'HM2'
joined.loc['62128','Mechanical_Support_Type'] = 'HM2'
joined.loc['PGD-2','Mechanical_Support_Type'] = 'HM2'
joined.loc['MATCH-2','Mechanical_Support_Type'] = 'HM2'
joined.loc['PGD-5','Mechanical_Support_Type'] = 'HM2'
joined.loc['MATCH-5','Mechanical_Support_Type'] = 'HM2'
joined.loc['PGD-11','Mechanical_Support_Type'] = 'TAH'
joined.loc['MATCH-11','Mechanical_Support_Type'] = 'TAH'
joined.loc['PGD-17','Mechanical_Support_Type'] = 'TAH'
joined.loc['MATCH-17','Mechanical_Support_Type'] = 'TAH'
joined.loc['Pi-18-1-128N','Mechanical_Support_Type'] = 'HM2'
joined.loc['Pi-18-4-129C','Mechanical_Support_Type'] = 'HM2'
joined.loc['Pi-26-2-130N','Mechanical_Support_Type'] = 'HM2'

In [ ]:
joined.to_csv(dir_+'mortality_X_y.csv')

## Summary

In [ ]:
joined = pd.read_csv('../../data/mortality_X_y.csv',index_col=0)

In [ ]:
demog = (
    joined.
    loc[:,
        ['Mechanical_Support_Y',
         'Mechanical_Support_Type',
         'PGD','expired',
         'Cohort_Columbia',
         'Cohort_Cedar',
         'Cohort_Paris',
         'graft_date',
         'expiration_date',
         'days_to_death',
        'deathdays']
       ]
)
demog

In [ ]:
display(demog.drop_duplicates().PGD.value_counts()) #correct with paper!
demog.drop_duplicates().Mechanical_Support_Y.value_counts() #correct with paper!

In [ ]:
display(demog.drop_duplicates().query('Mechanical_Support_Y==1')['Mechanical_Support_Type'].value_counts())
display(demog.drop_duplicates().groupby(['expired','Mechanical_Support_Type'])['graft_date'].count())
display(demog.drop_duplicates().groupby(['PGD','expired'])['graft_date'].count())

In [ ]:

display(
    demog.
    query('Cohort_Paris==1')
) 

There's 88 patients total. There were 46 patients that did not have PGD and 42 that had PGD. There were 21 patients that had an LVAD prior to their transplant. This agrees with the tables in the published paper (sanity check). There were 44 out of 46 that did not have PGD and didn't die after transplant. In contrast, there were 22 out of 42 that had PGD and eventually died.

We have recorded that 22 patients died after transplant (1 at Columbia, 14 at cedars, and 6 at paris). There were 5 out of 22 patients that had mechanical support prior to transplant. There were 14 out of 22 patients that died within a year; only 1 person had any mechanical support - they had an HM2 and died after 32 days. Everyone that died within a year had PGD. 

Out of 21 patients that had mechanical support, there were 3 patients at Columbia that had the recalled HW device, 14 (8 at Columbia, 4 at cedars, and 2 at paris) patients had the HM2 device, and 4 at Cedars had an artificial heart (TAH). Interestingly, 12 out of 14 that had an HM2 didn't die. There were 2 out of 4 patients that had a TAH didn't died and 2 out of 3 with a HW didn't died. 

| Var | No PGD | PGD |
| --- | ------ | --- |
| --- | N = 46 | N = 42|
| --- | ------ | --- |
| Died|   2 (no LVAD; 2 and 4.5 years later)   |  20 (2 TAH, 1 HW, and 2 HM2) |

In [ ]:
tmp = \
(
    demog.
    drop_duplicates().
    sort_values('days_to_death').
    loc[:,['PGD','Mechanical_Support_Type','days_to_death']].
    reset_index(drop=True)
)
tmp.head(40)

In [ ]:
print(joined[['expired','PGD','days_to_death','Age']].drop_duplicates().query('expired==1 & PGD==1')['days_to_death'].describe()/365)
print(joined[['expired','PGD','days_to_death','Age']].drop_duplicates().query('expired==1 & PGD==0')['days_to_death'].describe()/365)

In [ ]:
print(min(
    (pd.Timestamp.now().normalize() - pd.to_datetime(demog['graft_date'])) / 
    np.timedelta64(1, 'Y')
))
print(max(
    (pd.Timestamp.now().normalize() - pd.to_datetime(demog['graft_date'])) / 
    np.timedelta64(1, 'Y')
))
np.median((pd.Timestamp.now().normalize() - pd.to_datetime(demog['graft_date'])) / 
    np.timedelta64(1, 'Y'))

In [ ]:
(
    pd.Timestamp.now().normalize() - pd.to_datetime(joined.graft_date)
)  / pd.Timedelta(1, unit='d')

In [ ]:
Y_survival = (joined[['expired']]==0).astype(int)
Y_survival.columns = ['Survival']
Y_survival.index.name=''
display(Y_survival.value_counts())
Y_survival_wn_year = Y_survival.copy()
Y_survival_wn_year.loc[
    joined.index[joined.days_to_death>365],'Survival'
] = 1
display(Y_survival_wn_year.value_counts())

# Protein correlations between biological replicates

In [ ]:
proteins = pickle.load(open(dir_+'proteins_no_immunoglobulins.pkl','rb'))
X_all_proteins.loc[:,proteins]

In [ ]:
corr_df = \
X_all_proteins.loc[:,proteins].T.corr(method='spearman')
lowertribool = np.tril(np.ones(corr_df.shape)).astype(np.bool)
corr_df = corr_df.where(lowertribool)
corr_df_melt = corr_df.stack().rename_axis(['Sample1','Sample2']).reset_index()
corr_df_melt.columns = ['Sample1','Sample2','Value']
corr_df_melt = corr_df_melt.query('Sample1!=Sample2')
corr_df_melt

In [ ]:
bioreps = ['Pi-13-3-128N', 'Pi-13-4-126',
       'Pi-14-4-127C', 'Pi-14-4-128N', 'Pi-18-1-128N', 'Pi-18-4-129C',
       'Pi-19-1-129C', 'Pi-19-4-129N', 'Pi-20-1-129N',
       'Pi-20-4-130C', 'Pi-21-1-130C', 'Pi-21-4-130N']
diedbioreps = ['Pi-19-1-129C', 'Pi-19-4-129N']
notbioreps = np.setdiff1d(X_all_proteins.index,bioreps)

In [ ]:
corr_df_melt.query('Sample1 in @diedbioreps').sort_values('Value')

In [ ]:
print(corr_df_melt.query('Value>0.5').shape[0])
print(corr_df_melt.query('Sample1 in @bioreps & Value>0.5').shape[0])
print(corr_df_melt.shape[0])
corr_df_melt.query('Value>0.5').shape[0]/corr_df_melt.shape[0]

## Protein distribution (Figure S2)

In [ ]:
cumc = pd.read_csv('../../data/df_samples_cumc_allsets.csv',index_col=0)
cedar = pd.read_csv('../../data/df_samples_cedar_allsets.csv',index_col=0)
paris = pd.read_csv('../../data/df_samples_paris_allsets.csv',index_col=0)

In [ ]:
cumc_df = (cumc.
          rename_axis('Protein').
          loc[ig_proteins].
          apply(lambda x : (x - np.mean(x)) / np.std(x),axis=1).
          reset_index().
          melt(id_vars='Protein'))
cumc_df['Cohort'] = 'Columbia'

cedar_df = (cedar.
          rename_axis('Protein').
          loc[ig_proteins].
          apply(lambda x : (x - np.mean(x)) / np.std(x),axis=1).
          reset_index().
          melt(id_vars='Protein'))
cedar_df['Cohort'] = 'Cedar-Sinai'

paris_df = (paris.
          rename_axis('Protein').
          loc[ig_proteins].
          apply(lambda x : (x - np.mean(x)) / np.std(x),axis=1).
          reset_index().
          melt(id_vars='Protein'))
paris_df['Cohort'] = 'Pitíe Salpetriere'

In [ ]:
matplotlib.rcParams['axes.titlepad'] = 8
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['axes.labelsize'] = 16
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16
dpi=300

fig,ax = plt.subplots(nrows=3,ncols=1,sharex=True,sharey=True,dpi=dpi,figsize=(6,4))

cohorts=['Columbia','Cedar-Sinai','Pitíe Salpetriere']

for i,grp in cumc_df.groupby('variable'):
    sns.distplot(grp['value'],
                 color='Blue',
                 label=cohorts[0],
                 kde=False,
                 ax=ax[0])
    ax[0].set_alpha(0.8)

for i,grp in cedar_df.groupby('variable'):
    sns.distplot(grp['value'],
                 color='Green',
                 label=cohorts[1],
                 kde=False,
                 ax=ax[1])
    ax[1].set_alpha(0.8)

for i,grp in paris_df.groupby('variable'):
    sns.distplot(grp['value'],
                 color='red',
                 label=cohorts[2],
                 kde=False,
                 ax=ax[2])
    ax[2].set_alpha(0.8)
sns.despine()
ax[0].set_xlabel('')
ax[1].set_xlabel('')

for i,a in enumerate(ax):
    a.text(2.5,50,cohorts[i])
    a.set_xlim(-5,5)
ax[1].set_ylabel('Density')
ax[1].yaxis.set_label_coords(-0.1,0)
ax[0].set_title('Exosome protein expression distribution')
ax[2].set_xlabel('Standardized protein expression')

fig.tight_layout()
fig.savefig(dir_+'ProteinDescription_distributions.pdf')

## Protein distribution tests

In [ ]:
joined = pd.read_csv('../../data/mortality_X_y.csv',index_col=0)

In [ ]:
survived = (joined['expired']==0)
lst = []
for prot in X_all_proteins.columns:
    df = X_all_proteins.apply(lambda x : (x - min(x)) / (max(x) - min(x)),axis=0)[[prot]]
    dat = df.join(
        survived.map({True : 'Survived',False : 'Died'})
    ).join(joined.PGD.map({1 : 'PGD',0 : 'nonPGD'}))
    fig,ax=plt.subplots(figsize=(5,3),dpi=300)
    sns.boxplot('expired',prot,data=dat,color='lightgray',ax=ax)
    sns.swarmplot(
        'expired',prot,
        #hue='PGD',
        data=dat,
        color="black",
        edgecolor='black',ax=ax
    )
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticklabels(ax.get_yticks(),fontsize=12,weight='bold')
    ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.1f'))
    ax.set_xticklabels(ax.get_xticklabels(),fontsize=12,weight='bold')
    gene_log_sum = idmap_sub.Protein.isin([prot]).sum()
    if gene_log_sum>0:
        gene = idmap_sub.query('Protein==@prot')['Gene_name'].values[0]
    else:
        gene=''
    fig.savefig('../../docs/imgs/survival_protein_tests/'+prot+'_'+gene+'.png')
    plt.close()
    
    val = \
    ttest_ind(
        dat.query('expired=="Died"')[prot].values,
        dat.query('expired=="Survived"')[prot].values
    )[0]
    pgdval = \
    ttest_ind(
        dat.query('PGD=="nonPGD" & expired=="Died"')[prot].values,
        dat.query('PGD=="nonPGD" & expired=="Survived"')[prot].values
    )[0]
    lst.append([
        prot,val,pgdval,
        np.mean(dat.query('expired=="Survived"')[prot].values),
        np.mean(dat.query('PGD=="PGD" & expired=="Survived"')[prot].values),
        np.mean(dat.query('PGD=="nonPGD" & expired=="Survived"')[prot].values)
    ])

In [ ]:
(
    pd.DataFrame(
        lst,columns=["Protein","mean",
                     "mean_pgd_cov","avg",
                     "avg_pgd","avg_nonpgd"]
    ).
    query('mean_pgd_cov>mean & Protein in ["P00734","P08697","P00740",\
          "Q96IY4",]').
    eval('pi = (avg_nonpgd-avg)/avg')
)

## Differential analysis functions

In [ ]:
def balanced_resample(Y,seed=42):
	"""
	Given a binary pandas series, resample after balancing for equal size of classes
	"""
	
	Y.sort_values(inplace=True)
	num_to_sample = Y.value_counts().min()
	
	dfs = []
	for grp in Y.unique():
		y = Y[Y==grp].head(num_to_sample)
		dfs.append(resample(y,random_state=seed))  
	
	return pd.concat(dfs)

def pull_logit_coefficients(fit):
	
	return fit.coef_[0][0]

def coef_to_prob(coef):
	
	odds = np.exp(coef)
	prob = odds/(1 + odds)
	
	return prob

def coef_to_odds(coef):
	
	odds = np.exp(coef)
	
	return odds

def prediction(X,Y,model,seed=42):
	"""
	Given a feature matrix and binary class series, 
	balance then resample y (depends on balanced_resample),
	predict and grab logistic regression coefficients,
	convert and return probability.
	
	"""
	Y_balanced = resample(Y,random_state=seed)
	X_balanced = X.loc[Y_balanced.index]

	fit = model.fit(X_balanced,Y_balanced.values.reshape(-1,1))

	coef = pull_logit_coefficients(fit)
	
	return coef_to_odds(coef)

def balanced_prediction(X,Y,model,seed=42):
	"""
	Given a feature matrix and binary class series, 
	balance then resample y (depends on balanced_resample),
	predict and grab logistic regression coefficients,
	convert and return probability.
	
	"""
	Y_balanced = balanced_resample(Y,seed=seed)
	X_balanced = X.loc[Y_balanced.index]
	
	fit = model.fit(X_balanced,Y_balanced)
	
	coef = pull_logit_coefficients(fit)
	
	return coef_to_odds(coef)

def bootstrap_prediction_transformations(odds_boot,var='variable'):
	df = pd.DataFrame([
		[key for key in odds_boot.keys()],
		[np.median(odds_boot[key]) for key in odds_boot.keys()]
	],
		index=[var,'bootstrap_median']
	).T
	sorted_df = df.sort_values(['bootstrap_median'],ascending=[False])	
	output = (pd.DataFrame.from_dict(odds_boot).
			  reset_index().rename(columns={'index' : 'bootstrap'}).
			  melt(id_vars='bootstrap',var_name=var,value_name='odds').
			  set_index(var).
			  join(sorted_df.set_index(var))
			 )
	odds_wcov_boot = output.reset_index().copy()
	variables = odds_wcov_boot[var].unique()
	err = {}
	for p in variables:
		q = '{} == "{}"'.format(var,p)
		lwr = odds_wcov_boot.query(q).odds.quantile(.025)
		mean = odds_wcov_boot.query(q).odds.mean()
		median = odds_wcov_boot.query(q).odds.quantile(.5)
		upr =odds_wcov_boot.query(q).odds.quantile(.975)
		err[p] = [lwr,mean,median,upr]
	err_df = pd.DataFrame(err,index=['lwr','mean','median','upr']).T.rename_axis(var)
	return output, err_df

from joblib import Parallel, delayed

def bootstrap_of_fcn(func=None,params={},n_jobs=4,nboot=2):	
	if func==None:
		return "Need fcn to bootstrap"
	parallel = Parallel(n_jobs=n_jobs)
	return parallel(delayed(func)(seed=k,**params) for k in range(nboot))


## Survival multivariate logistic regression

In [ ]:
cohort_covs = pd.read_csv('../../data/integrated_cohort_covariates_minus_paris.csv',index_col=0)
proteins = pickle.load(open('../../data/proteins_immunoglobulins.pkl','rb'))

In [ ]:
C=1
seed = 42
tol=1e-3

model = {"Logistic Regression" : 
linear_model.LogisticRegression(
	C=C,
	penalty='l1',
	solver="liblinear",
	random_state=seed,fit_intercept=True
	)
		 }


In [ ]:
joined = pd.read_csv(dir_+'mortality_X_y.csv',index_col=0)
X = X_all_proteins.copy().apply(lambda x : (x - x.min()) / (x.max() - x.min()))
Y = joined[['expired']][['expired']].copy()
Y = Y.loc[X.index.values]
Y = (Y==0).astype(int)
Y.columns = ['Survival']

In [ ]:
X_all_clinical.columns

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
vars_=\
['Age','BMI','CVP','CVP/PCWP','Creatinine',
 'INR','Ischemic_Time','Blood_Type_A','Blood_Type_B',
'Blood_Type_O',
 #'Blood_Type_AB',
 'Donor_Age',
#'Radial_Score',
 'Sodium','MELDXI',
'PA_Diastolic','PA_Mean','PA_Systolic','PCWP','TBILI',
'Antiarrhythmic_Use_Y','Beta_Blocker_Y','Cardiomyopathy_Adriamycin',
'Cardiomyopathy_Amyloid',
 #'Cardiomyopathy_Chagas',
 'Cardiomyopathy_Congenital',
#'Cardiomyopathy_Hypertrophic cardiomyopathy','Cardiomyopathy_Idiopathic',
'Cardiomyopathy_Ischemic','Cardiomyopathy_Myocarditis',
#'Cardiomyopathy_Valvular Heart Disease','Cardiomyopathy_Viral',
'Diabetes_Y','History_Of_Tobacco_Use_Y','Mechanical_Support_Y',
'Prior_Inotrope_Y','PGD'
 #'Sex_F'
]

In [ ]:
samples_clean = [x[0] for x in X_all_clinical.index.str.split('-[1234]-1[23][NC]{0,1}')]
len(np.unique(samples_clean))
X = joined[vars_]
X.index = samples_clean
X = X.drop_duplicates()

In [ ]:
y = Y.copy()
y.index = samples_clean
y = y.reset_index().drop_duplicates().set_index('index')
y

In [ ]:
X.columns

In [ ]:
log_reg = sm.Logit(y,X).fit()
#https://stackoverflow.com/questions/51734180/converting-statsmodels-summary-object-to-pandas-dataframe
multi = pd.read_html(log_reg.summary().tables[1].as_html(), header=0, index_col=0)[0]
multi.columns=['coef','std_err','z','pvalue','lwr','upr']
print(multi.shape)
display(multi.sort_values('pvalue'))
print(multi.query('pvalue<0.05').shape)
print(multi.query('lwr>0 | upr<0').shape)

## GSEA rank statistic (univariate, bootstrapped logistic regression)

In [ ]:
cohort_covs = pd.read_csv('../../data/integrated_cohort_covariates_minus_paris.csv',index_col=0)
proteins = pickle.load(open('../../data/proteins_immunoglobulins.pkl','rb'))

In [ ]:
joined = pd.read_csv(dir_+'mortality_X_y.csv',index_col=0)
X = X_all_proteins.copy().apply(lambda x : (x - x.min()) / (x.max() - x.min()))
Y = joined[['expired']][['expired']].copy()
Y = Y.loc[X.index.values]
Y = (Y==0).astype(int)
Y.columns = ['Survival']

In [ ]:
prots = X.columns.values
lst = []
for prot in prots:
    print(prot)
    if idmap_sub.query('Protein==@prot').shape[0]==0: continue
    gene = idmap_sub.query('Protein==@prot').Gene_name.values[0]

    x = X_all_proteins.loc[Y[(Y==1).values].index.values,prot].values
    y = X_all_proteins.loc[Y[(Y==0).values].index.values,prot].values

    fig,ax=plt.subplots(dpi=300)
    sns.swarmplot(
        'Survival',
        'expr',
        data=pd.concat(
        [
            pd.DataFrame({'expr' : x,'Survival' : 'Survived'}),
            pd.DataFrame({'expr' : y,'Survival' : 'Died'})
        ])
    )
    ax.set_xlabel(None)
    ax.set_ylabel('Protein expression')
    ax.set_title(
        gene+
        '\nT test statistic = '+str(np.round(ttest_ind(x,y)[0],3))+
        '; Mann Whitney Pvalue = '+str(np.round(mannwhitneyu(x,y)[1],3))
    )
    fig.savefig('../../docs/imgs/survival_protein_tests/'+gene+'_test.png')
    plt.close()
    lst.append([mannwhitneyu(x,y)[0]/(len(x)*len(y)),np.var(x),np.var(y),gene])

In [ ]:
tmp = pd.DataFrame(lst,columns=['AUROC','svar','dvar','Gene_name']).sort_values('AUROC')
tmp['IG'] = tmp.Gene_name.str.match('^IG')
a = tmp.query('IG==True').sample(1000,replace=True).AUROC.values
b = tmp.query('IG==False').sample(1000,replace=True).AUROC.values
print(mannwhitneyu(a,b))
a = tmp.query('IG==True').sample(1000,replace=True).svar.values
b = tmp.query('IG==False').sample(1000,replace=True).svar.values
print(np.mean(a))
print(np.mean(b))
print(mannwhitneyu(a,b))
a = tmp.query('IG==True').sample(1000,replace=True).dvar.values
b = tmp.query('IG==False').sample(1000,replace=True).dvar.values
print(np.mean(a))
print(np.mean(b))
print(mannwhitneyu(a,b))

In [ ]:
C=1
seed = 42
tol=1e-3

model = {"Logistic Regression" : 
linear_model.LogisticRegression(
	C=C,
	penalty='l1',
	solver="liblinear",
	random_state=seed,fit_intercept=True
	)
		 }


In [ ]:
joined = pd.read_csv('../../data/mortality_X_y.csv',index_col=0)
X = X_all_proteins.copy().apply(lambda x : (x - x.min()) / (x.max() - x.min()))
Y = joined[['expired']][['expired']].copy()
Y = Y.loc[X.index.values]

In [ ]:
cohort_covs = pd.read_csv('../../data/integrated_cohort_covariates_minus_paris.csv',index_col=0)
proteins = pickle.load(open('../../data/proteins_immunoglobulins.pkl','rb'))

In [ ]:
C=1
seed = 42
tol=1e-3

model = {"Logistic Regression" : 
linear_model.LogisticRegression(
	C=C,
	penalty='l1',
	solver="liblinear",
	random_state=seed,fit_intercept=True
	)
		 }


In [ ]:
joined = pd.read_csv('../../data/mortality_X_y.csv',index_col=0)
X = X_all_proteins.copy().apply(lambda x : (x - x.min()) / (x.max() - x.min()))

Y = joined[['expired']][['expired']].copy()
Y = Y.loc[X.index.values]
Y = (Y==0).astype(int)
Y.columns = ['Survival']
Y

In [ ]:
nboot=200

boots = []
for prot in proteins:
	params = {
	'X' : X[[prot]].join(cohort_covs),
	'Y' : Y,
	'model' : model['Logistic Regression']}
	lst = bootstrap_of_fcn(func=prediction,params=params,n_jobs=num_cores,nboot=nboot)
	boots.append(lst)

odds_boot = {}
for i,prot in enumerate(proteins):
	odds_boot[prot] = boots[i]

output, err_df = bootstrap_prediction_transformations(odds_boot)


In [ ]:

dat = \
(
    output.
    reset_index().
    groupby('variable')['odds'].
    mean().
    sort_values().
    reset_index().
    set_index('variable').
    join(
        X_all_proteins.
        loc[Y.index[Y.Survival==1]].
        reset_index().
        melt(id_vars='index').
        groupby('variable')['value'].
        mean().
        reset_index().
        set_index('variable')
    ).
    join(
    idmap_sub.set_index('Protein'))
)
fig,ax = plt.subplots(dpi=300)
sns.scatterplot('odds','value',data=dat,ax=ax)
ax.set_ylabel('Average protein expression\nfrom patients who died')
ax.set_xlabel('Average odds association to mortality')
display(dat.query('odds<0.5'))
display(dat.query('odds>2'))

In [ ]:
fig,ax = plt.subplots(dpi=300,figsize=(12,4))
ax = output.reset_index().groupby('variable')['odds'].mean().sort_values().plot(ax=ax)

In [ ]:
univariate = dat.copy()
univariate[['model']] = 'univariate'
univariate

In [ ]:
(
    univariate.
    reset_index().
    set_index('Gene_name').
    loc[:,['odds','variable']].
    sort_values('odds',ascending=False).
    drop_duplicates().
    to_csv(dir_+'survival_rank_statistic.csv')
)

## Prediction processing functions

In [ ]:
def generate_val_scores(ppred_df,
                        scorer = { 
                            'roc_auc' : metrics.roc_auc_score,  
                            'ppv' : metrics.average_precision_score,
                            'npv' : metrics.average_precision_score
                        }):
    bin_names = ['accuracy']
    n_names = ['npv','specificity']
    score_names = scorer.keys()
    vals = []
    for set_ in ppred_df.index.unique():
        tmp = ppred_df.loc[set_]
        for b in range(50):
            sub = tmp.sample(tmp.shape[0],replace=True,random_state=b)
            arr = \
            [scorer[x](sub.y_true,sub.y_proba) 
             if x not in n_names else scorer[x](sub.y_true,sub.y_proba,pos_label=0) 
             for x in score_names]
            arr.extend([set_,b])
            vals.append(
                arr
            )
    cols = [x for x in score_names]
    cols.extend(['set','bootstrap'])
    val_df = \
    (
        pd.DataFrame(
            vals,
            columns=cols
        ).
        groupby('set')[[x for x in score_names]].
        describe(
            percentiles=[0.025,0.975]
        )
    )
    val_df.columns = \
    [x[0]+'_'+x[1].replace('2.5%','lwr').replace('97.5%','upr') for x in val_df.columns]
    return (
        val_df.
        loc[:,np.concatenate([[x+'_lwr',x+'_mean',x+'_upr'] for x in score_names])]
    )
def get_pperf_roc_curve_stats(dat,n=50):
    
    tups = []
    for b in range(n):
        x = (dat.
             sample(n=dat.shape[0],replace=True,random_state=b)
            )
        f,t,th = roc_curve(x.y_true,x.y_proba)

        tups.append(
            pd.DataFrame({ 'fpr' : f,
                          'tpr' : t,
                          't' : th
                         }
                        )
        )

    tmp = pd.concat(tups).groupby('t').mean()
    fpr = tmp['fpr'].values
    tpr = tmp['tpr'].values
    return fpr,tpr
    
def get_pperf_precision_recall_curve_stats(dat,n=50):
    
    tups = []
    for b in range(n):
        x = (dat.
             sample(n=dat.shape[0],replace=True,random_state=b)
            )
        r,p,th = precision_recall_curve(x.y_true,x.y_proba)
        r = list(r)
        p = list(p)
        r.pop()
        p.pop()
        tups.append(
            pd.DataFrame({ 'precision' : p,
                          'recall' : r,
                          't' : th
                         }
                        )
        )

    tmp = pd.concat(tups).groupby('t').mean()
    p = tmp['precision'].tolist()
    r = tmp['recall'].tolist()
    p[0] = 1
    r[0] = 0
    return p,r

def plt_atts_roc(ax,fig):

    ax.set_xlim(-0.01,1.01)
    ax.set_ylim(-0.01,1.01)

    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)

    #ax.set_ylabel('Sensitivity',size=18)
    #ax.set_xlabel('1 - Specificity',size=18)
    ax.tick_params(axis='both', which='major', labelsize=14)

    fig.tight_layout()
    
    return fig

def plt_atts_pr(ax,fig):

    ax.set_xlim(-0.01,1.01)
    ax.set_ylim(-0.01,1.01)

    lims = [
        [np.min(ax.get_xlim()), np.max(ax.get_ylim())],  
        [np.max(ax.get_xlim()), np.min(ax.get_ylim())]
    ]

    # now plot both limits against eachother
    ax.plot(lims[0], lims[1], 'r--', alpha=0.75, zorder=0)

    #ax.set_ylabel('Precision',size=18)
    #ax.set_xlabel('Recall',size=18)
    ax.tick_params(axis='both', which='major', labelsize=14)

    fig.tight_layout()
    
    return fig

## Survival predictions and Volcano plot (Figure 2)

In [ ]:
ppred_df = pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions_survival.csv',
                index_col=0)
arr_dfs = []
for set_,tmp in ppred_df.groupby('set'):
    vals = []
    for b in range(50):
        sub = tmp.sample(tmp.shape[0],replace=True,random_state=b)
        conf = metrics.confusion_matrix(sub.y_true,sub.y_pred)
        tn = conf[0][0]
        tp = conf[1][1]
        fn = conf[1][0]
        fp = conf[0][1]
        auc = metrics.roc_auc_score(sub.y_true,sub.y_proba)
        vals.append([auc,tp,tn,fp,fn,
                     np.mean(sub.y_proba[sub.y_true==1]),
                     np.mean(sub.y_proba[sub.y_true==0])])
    arr = np.array(vals)
    arr_df = \
    pd.DataFrame(
        np.mean(arr,0),
        index=['AUROC','TP','TN','FP','FN','mean_case_proba','mean_ctrl_proba'],
        columns=[set_]
    ).T
    arr_dfs.append(
        arr_df.eval(
            '''
            sensitivity = (TP/(TP+FN))
            PPV = (TP/(TP+FP))
            specificity = (TN/(TN+FP))
            NPV = (TN/(TN+FN))
            '''
        )
    )

perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance_survival.csv'
)
perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in perf_df.set_features]

tmp = (
    pd.
    concat(arr_dfs).
    join(
        perf_df.
        set_index('set').
        loc[:,['set_features']]
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.
        set_index('Protein')
    ).
    sort_values('AUROC')
)
tmp

In [ ]:
tmp.sort_values('AUROC').tail(10)

In [ ]:
perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance_survival.csv'
)
perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in perf_df.set_features]
perf_df = \
(
    perf_df.
    loc[:,['set','set_features']].
    set_index('set').
    join(
        generate_val_scores(
            pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions_survival.csv',
                index_col=0).set_index('set')
        )
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('roc_auc_mean')
)

fimp_df = pd.read_csv(dir_+'mortality_predictions_marker_feature_importance_survival.csv')
fimp_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in fimp_df.set_features]

In [ ]:
(
    perf_df.
    loc[:,['Gene_name','roc_auc_mean','ppv_mean','npv_mean']].
    sort_values('npv_mean')
)

In [ ]:
full_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_feature_importance_survival.csv',
    index_col=0
)
full_perm_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_permuted_feature_importance_survival.csv',
    index_col=0
)
proteins = np.intersect1d(full_fimp_df.Feature.unique(),full_perm_fimp_df.Feature.unique())
sig_lst = []
for prot in proteins:
    x = full_fimp_df[full_fimp_df.Feature==prot].Importance.values
    y = full_perm_fimp_df[full_fimp_df.Feature==prot].Importance.values
    lwr, upr = (
        full_perm_fimp_df[
            full_fimp_df.Feature==prot
        ].
        Importance.
        describe(
            percentiles=[0.025,0.975]
        ).
        loc[['2.5%','97.5%']]
    )
    stat, pv = ks_2samp(x,y)
    sig_lst.append(
        pd.DataFrame(
            {'statistic' : stat,'pvalue' : pv,'perm_fimp_lwr' : lwr,'perm_fimp_upr' : upr},
            index=[prot]
        )
    )
sig_df = pd.concat(sig_lst)
sig_df['padj'] = multipletests(sig_df['pvalue'],method='bonferroni')[1]
sig_df

In [ ]:
dat = \
(
    perf_df.
    loc[:,['roc_auc_mean','roc_auc_lwr','roc_auc_upr','set','Gene_name']].
    rename(
        columns={
            'roc_auc_mean' : 'perf',
            'roc_auc_lwr' : 'perf_lwr',
            'roc_auc_upr' : 'perf_upr'
        }
    ).
    reset_index().
    set_index('set').
    join(
        fimp_df.
        query('Feature!="Intercept"').
        set_index('set').
        loc[:,['mean','2.5%','97.5%']].
        rename(columns={'mean' : 'fimp','2.5%' : 'fimp_lwr','97.5%' : 'fimp_upr'})
    ).
    set_index('set_features').
    join(sig_df)
)
dat['-log10pvalue'] = -np.log10(dat['pvalue'])

dat['Marker'] = 'blue'
dat.loc[dat.dropna().index.values,'Marker'] = 'red'

dat = dat.query('set_features!="PGD"')
dat = dat.query('set_features!="expired"')
dat = dat.query('set_features!="Survival"')

dat.to_csv(dir_+'mortality_survival_marker_predictions.csv')

display(dat.sort_values('perf').tail(10))
display(dat.sort_values('fimp').head(10))
display(dat.sort_values('fimp').tail(10))

palette = 'RdBu_r'
fig,ax = plt.subplots(dpi=300)
plot = plt.scatter(dat['fimp'].values,
          dat['perf'].values,
          c=dat['-log10pvalue'].values,
          cmap=palette)

plt.clf()
plt.colorbar(plot)

ax = sns.scatterplot('fimp','perf',hue='-log10pvalue',data=dat,
                    style='Marker',edgecolor='k',
                   palette=palette)
ax.set_xlabel(r'$\beta$ coefficient',size=20)
ax.set_ylabel('AUROC',size=20)
ax.legend_.remove()

fig.tight_layout()

fig.savefig('../../docs/imgs/mortality_mccv_predictions_survival.png')

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<0 & perm_fimp_upr>0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    ).
    loc[:,
        ['perf_lwr',
         'perf','perf_upr','fimp_lwr',
         'fimp','fimp_upr','padj'
        ]
       ].
    sort_values('fimp',ascending=False)
)

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    )
)

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    )
).shape

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    ).
    loc[:,
        ['perf_lwr',
         'perf','perf_upr','fimp_lwr',
         'fimp','fimp_upr','padj','Gene_name'
        ]
       ].
    sort_values('fimp',ascending=False)
).to_csv(dir_+'mortality_significant_prediction_stats_survival.csv')
(
    dat.
    reset_index().
    query('fimp_lwr>0').
    loc[:,
        ['set_features','perf_lwr',
         'perf','perf_upr','fimp_lwr',
         'fimp','fimp_upr'
        ]
       ].
    set_index('set_features').
    sort_values('fimp',ascending=False)
).to_csv(dir_+'mortality_prediction_stats.csv')

In [ ]:
Y = joined[['expired']][['expired']].copy()
Y = (Y==0).astype(int)
Y.columns = ['Survival']
Y

In [ ]:
ppred_df = \
(
    pd.read_csv(
        dir_+
        'mortality_predictions_marker_patient_predictions_survival.csv',
        index_col=0).
    set_index('set').
    join(
        fimp_df[['set','Feature']].
        query('Feature!="Intercept"').
        drop_duplicates().
        set_index('set')
    ).
    reset_index().
    set_index('Feature').
    join(idmap_sub.set_index('Protein'))
)
tmp = ppred_df[['Sample','y_proba']].set_index('Sample').join(Y)
tmp = \
pd.concat([
    pd.DataFrame(
        [tmp.query('Survival==1').sample(100).y_proba.values,np.repeat('Survived',100)]
    ).T,
    pd.DataFrame(
        [tmp.query('Survival==0').sample(100).y_proba.values,np.repeat('Died',100)]
    ).T
])
tmp.columns = ['y_proba','Survival']
tmp.y_proba = tmp.y_proba.astype(float)
display(tmp)
fig,ax = plt.subplots(dpi=300)
sns.kdeplot(data=tmp,x='y_proba',hue='Survival',ax=ax)

In [ ]:
a = tmp.query('Survival=="Survived"').y_proba.values
b = tmp.query('Survival=="Died"').y_proba.values
ttest_ind(a,b)

In [ ]:
dpi=300
feats=dat.query('perf>0.6').index.values
for feat in feats:
    gene = idmap_sub.query('Protein==@feat')['Gene_name'].values[0]
    
    pperf=ppred_df.loc[feat]
    c='black'
    
    func=get_pperf_roc_curve_stats
    fpr,tpr = func(pperf)

    fig,ax = plt.subplots(dpi=dpi)
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.00001)
    ax.set_xlabel('')
    fig = plt_atts_roc(ax,fig)
    fig.savefig('../../docs/imgs/'+feat+'_'+gene+'_roc_curve.png')
    
    func=get_pperf_precision_recall_curve_stats
    fpr,tpr = func(pperf)

    fig,ax = plt.subplots(dpi=dpi)
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.00001)
    fig = plt_atts_pr(ax,fig)
    fig.savefig('../../docs/imgs/'+feat+'_'+gene+'_pr_curve.png')

### Comparison to predictions with covariates

In [ ]:
s_perf_df = perf_df
s_fimp_df = fimp_df

In [ ]:
swcovs_perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance_survival_wcovs.csv'
)
swcovs_perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in swcovs_perf_df.set_features]
swcovs_perf_df = \
(
    swcovs_perf_df.
    loc[:,['set','set_features']].
    set_index('set').
    join(
        generate_val_scores(
            pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions_survival_wcovs.csv',
                index_col=0).set_index('set')
        )
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('roc_auc_mean')
)

swcovs_fimp_df = pd.read_csv(dir_+'mortality_predictions_marker_feature_importance_survival_wcovs.csv')
swcovs_fimp_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in swcovs_fimp_df.set_features]

In [ ]:
swcovs_perf_df[['set']]

In [ ]:
tmp = \
(
    swcovs_perf_df.
    query('set!=40 & set!=42 & set!=35').
    loc[:,['roc_auc_mean']].
    rename(columns = {'roc_auc_mean' : 'wcovs'}).
    join(
        s_perf_df.
        loc[:,['roc_auc_mean']].
        rename(columns = {'roc_auc_mean' : 'wocovs'})
    )
)
fig,ax=plt.subplots(dpi=300)
g = sns.scatterplot('wcovs','wocovs',data=tmp,ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),fontsize=12,weight='bold')
ax.set_xlabel('Prediction with site-of-origin covariates',fontsize=12,weight='bold')
ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.2f'))
ax.set_ylabel('Marker prediction',fontsize=12,weight='bold')
ax.set_yticklabels(ax.get_yticks(),fontsize=12,weight='bold')
ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.2f'))

x0, x1 = g.axes.get_xlim()
y0, y1 = g.axes.get_ylim()
lims = [max(x0, y0), min(x1, y1)]
g.axes.plot(lims, lims, 'r--')
fig.tight_layout()
fig.savefig('../../docs/imgs/survival_marker_predictions_w_wo_covariates.png')

In [ ]:
tmp = \
(
    s_fimp_df.
    query('Feature!="Intercept" & Feature!="Cohort_Columbia" & Feature!="Cohort_Cedar" & Feature!="Cohort_Paris"').
    set_index('set').
    loc[:,['mean']].
    rename(columns={'mean' : 'wcovs'}).
    join(
        swcovs_fimp_df.
        query('Feature!="expired" & Feature!="Survival" & Feature!="PGD"').
        query('Feature!="Intercept" & Feature!="Cohort_Columbia" & Feature!="Cohort_Cedar" & Feature!="Cohort_Paris"').
        set_index('set').
        loc[:,['mean']].
        rename(columns={'mean' : 'wocovs'})
    )
)
fig,ax=plt.subplots(dpi=300)
g = sns.scatterplot('wcovs','wocovs',data=tmp,ax=ax)
ax.set_xlabel('Marker association with site-of-origin covariates',fontsize=12,weight='bold')
ax.set_xticklabels(ax.get_xticks(),fontsize=12,weight='bold')
ax.set_ylabel('Marker association',fontsize=12,weight='bold')
ax.set_yticklabels(ax.get_yticks(),fontsize=12,weight='bold')

x0, x1 = g.axes.get_xlim()
y0, y1 = g.axes.get_ylim()
lims = [max(x0, y0), min(x1, y1)]
g.axes.plot(lims, lims, 'r--')
fig.tight_layout()
fig.savefig('../../docs/imgs/survival_marker_associations_w_wo_covariates.png')

In [ ]:
swcovs_full_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_feature_importance_survival_wcovs.csv',
    index_col=0
)
swcovs_full_perm_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_permuted_feature_importance_survival_wcovs.csv',
    index_col=0
)
proteins = np.intersect1d(swcovs_full_fimp_df.Feature.unique(),swcovs_full_perm_fimp_df.Feature.unique())
sig_lst = []
for prot in proteins:
    x = swcovs_full_fimp_df[swcovs_full_fimp_df.Feature==prot].Importance.values
    y = swcovs_full_perm_fimp_df[swcovs_full_fimp_df.Feature==prot].Importance.values
    lwr, upr = (
        swcovs_full_perm_fimp_df[
            swcovs_full_fimp_df.Feature==prot
        ].
        Importance.
        describe(
            percentiles=[0.025,0.975]
        ).
        loc[['2.5%','97.5%']]
    )
    stat, pv = ks_2samp(x,y)
    sig_lst.append(
        pd.DataFrame(
            {'statistic' : stat,'pvalue' : pv,'perm_fimp_lwr' : lwr,'perm_fimp_upr' : upr},
            index=[prot]
        )
    )
swcovs_sig_df = pd.concat(sig_lst)
swcovs_sig_df['padj'] = multipletests(swcovs_sig_df['pvalue'],method='bonferroni')[1]
swcovs_sig_df

In [ ]:
swcovs_dat = \
(
    swcovs_perf_df.
    loc[:,['roc_auc_mean','roc_auc_lwr','roc_auc_upr','set','Gene_name']].
    rename(
        columns={
            'roc_auc_mean' : 'perf',
            'roc_auc_lwr' : 'perf_lwr',
            'roc_auc_upr' : 'perf_upr'
        }
    ).
    reset_index().
    set_index('set').
    join(
        swcovs_fimp_df.
        query('Feature!="Intercept" & Feature!="Cohort_Columbia" & Feature!="Cohort_Cedar" & Feature!="Cohort_Paris"').
        set_index('set').
        loc[:,['mean','2.5%','97.5%']].
        rename(columns={'mean' : 'fimp','2.5%' : 'fimp_lwr','97.5%' : 'fimp_upr'})
    ).
    set_index('set_features').
    join(swcovs_sig_df)
)

In [ ]:
sig = (
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    ).
    sort_values('perf')
).index.unique()
print(len(sig))
sig_wcovs = (
    swcovs_dat.
    query(
        'perf>0.5 & (fimp_lwr<=0 & fimp_upr>=0)'
    ).
    sort_values('perf')
).index.unique()
len(sig_wcovs)

In [ ]:
np.setdiff1d(sig,sig_wcovs) #not significantly associated after covariates

## Survival prediction with PGD covariate

In [ ]:
perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance_survival_wpgdcov.csv'
)
perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in perf_df.set_features]
perf_df = \
(
    perf_df.
    loc[:,['set','set_features']].
    set_index('set').
    join(
        generate_val_scores(
            pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions_survival_wpgdcov.csv',
                index_col=0).set_index('set')
        )
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('roc_auc_mean')
)

fimp_df = pd.read_csv(dir_+'mortality_predictions_marker_feature_importance_survival_wpgdcov.csv')
fimp_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in fimp_df.set_features]

In [ ]:
full_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_feature_importance_survival_wpgdcov.csv',
    index_col=0
)
full_perm_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_permuted_feature_importance_survival_wpgdcov.csv',
    index_col=0
)
proteins = np.intersect1d(full_fimp_df.Feature.unique(),full_perm_fimp_df.Feature.unique())
sig_lst = []
for prot in proteins:
    x = full_fimp_df[full_fimp_df.Feature==prot].Importance.values
    y = full_perm_fimp_df[full_fimp_df.Feature==prot].Importance.values
    lwr, upr = (
        full_perm_fimp_df[
            full_fimp_df.Feature==prot
        ].
        Importance.
        describe(
            percentiles=[0.025,0.975]
        ).
        loc[['2.5%','97.5%']]
    )
    stat, pv = ks_2samp(x,y)
    sig_lst.append(
        pd.DataFrame(
            {'statistic' : stat,'pvalue' : pv,'perm_fimp_lwr' : lwr,'perm_fimp_upr' : upr},
            index=[prot]
        )
    )
sig_df = pd.concat(sig_lst)
sig_df['padj'] = multipletests(sig_df['pvalue'],method='bonferroni')[1]
sig_df

In [ ]:
dat = \
(
    perf_df.
    loc[:,['roc_auc_mean','roc_auc_lwr','roc_auc_upr','set','Gene_name']].
    rename(
        columns={
            'roc_auc_mean' : 'perf',
            'roc_auc_lwr' : 'perf_lwr',
            'roc_auc_upr' : 'perf_upr'
        }
    ).
    reset_index().
    set_index('set').
    join(
        fimp_df.
        query('Feature!="Intercept" & Feature!="PGD"').
        set_index('set').
        loc[:,['mean','2.5%','97.5%']].
        rename(columns={'mean' : 'fimp','2.5%' : 'fimp_lwr','97.5%' : 'fimp_upr'})
    ).
    set_index('set_features').
    join(sig_df)
)
dat = dat.query('set_features!="PGD"')
dat = dat.query('set_features!="expired"')
dat = dat.query('set_features!="Survival"')

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    )
)

In [ ]:
prots = \
(
    pd.
    read_csv(
        dir_+'mortality_significant_prediction_stats_survival.csv'
    ).
    loc[:,'Gene_name'].
    values
)

In [ ]:
(
    dat.
    query(
        '(perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & \
        (fimp_lwr>0 | fimp_upr<0) & padj<0.0001) | \
        Gene_name in @prots'
    ).
    loc[:,
        ['perf_lwr',
         'perf','perf_upr','fimp_lwr',
         'fimp','fimp_upr','padj','Gene_name'
        ]
       ].
    sort_values('fimp',ascending=False)
).to_csv(dir_+'mortality_significant_prediction_stats_survival_wpgdcov.csv')

## Survival <1 year predictions

In [ ]:
perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance_survival_wn_year.csv'
)
perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in perf_df.set_features]
perf_df = \
(
    perf_df.
    loc[:,['set','set_features']].
    set_index('set').
    join(
        generate_val_scores(
            pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions_survival_wn_year.csv',
                index_col=0).set_index('set')
        )
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('roc_auc_mean')
)

fimp_df = pd.read_csv(dir_+'mortality_predictions_marker_feature_importance_survival_wn_year.csv')
fimp_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in fimp_df.set_features]

In [ ]:
full_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_feature_importance_survival_wn_year.csv',
    index_col=0
)
full_perm_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_permuted_feature_importance_survival_wn_year.csv',
    index_col=0
)
proteins = np.intersect1d(full_fimp_df.Feature.unique(),full_perm_fimp_df.Feature.unique())
sig_lst = []
for prot in proteins:
    x = full_fimp_df[full_fimp_df.Feature==prot].Importance.values
    y = full_perm_fimp_df[full_fimp_df.Feature==prot].Importance.values
    lwr, upr = (
        full_perm_fimp_df[
            full_fimp_df.Feature==prot
        ].
        Importance.
        describe(
            percentiles=[0.025,0.975]
        ).
        loc[['2.5%','97.5%']]
    )
    stat, pv = ks_2samp(x,y)
    sig_lst.append(
        pd.DataFrame(
            {'statistic' : stat,'pvalue' : pv,'perm_fimp_lwr' : lwr,'perm_fimp_upr' : upr},
            index=[prot]
        )
    )
sig_df = pd.concat(sig_lst)
sig_df['padj'] = multipletests(sig_df['pvalue'],method='bonferroni')[1]
sig_df

In [ ]:
dat = \
(
    perf_df.
    loc[:,['roc_auc_mean','roc_auc_lwr','roc_auc_upr','set','Gene_name']].
    rename(
        columns={
            'roc_auc_mean' : 'perf',
            'roc_auc_lwr' : 'perf_lwr',
            'roc_auc_upr' : 'perf_upr'
        }
    ).
    reset_index().
    set_index('set').
    join(
        fimp_df.
        query('Feature!="Intercept" & Feature!="PGD"').
        set_index('set').
        loc[:,['mean','2.5%','97.5%']].
        rename(columns={'mean' : 'fimp','2.5%' : 'fimp_lwr','97.5%' : 'fimp_upr'})
    ).
    set_index('set_features').
    join(sig_df)
)
dat = dat.query('set_features!="PGD"')
dat = dat.query('set_features!="expired"')
dat = dat.query('set_features!="Survival"')

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    )
)

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perm_fimp_lwr<=0 & perm_fimp_upr>=0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    ).
    loc[:,
        ['perf_lwr',
         'perf','perf_upr','fimp_lwr',
         'fimp','fimp_upr','padj','Gene_name'
        ]
       ].
    sort_values('fimp',ascending=False)
).to_csv(dir_+'mortality_significant_prediction_stats_survival_wn_year.csv')

## Mortality lvad covariate predictions

In [ ]:
perf_df = \
pd.read_csv(
    dir_+
    'mortality_predictions_marker_performance_lvad_covariate.csv'
)
perf_df.set_features = [x.replace("['",'').replace("', 'lvad']",'').strip(", ") for x in perf_df.set_features]
fimp_df = pd.read_csv(dir_+'mortality_predictions_marker_feature_importance_lvad_covariate.csv')
fimp_df.set_features = [x.replace("['",'').replace(", 'lvad']",'').strip(", ") for x in fimp_df.set_features]

In [ ]:
perf_df.sort_values('mean')

In [ ]:
(
    perf_df.
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('mean').
    tail(20)
)

In [ ]:
(
    fimp_df.
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('mean').
    query('`2.5%`>0')
)

## PGD predictions

ppred_df = pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions_pgd.csv',
                index_col=0)
arr_dfs = []
for set_,tmp in ppred_df.groupby('set'):
    vals = []
    for b in range(50):
        sub = tmp.sample(tmp.shape[0],replace=True,random_state=b)
        conf = metrics.confusion_matrix(sub.y_true,sub.y_pred)
        tn = conf[0][0]
        tp = conf[1][1]
        fn = conf[1][0]
        fp = conf[0][1]
        auc = metrics.roc_auc_score(sub.y_true,sub.y_proba)
        vals.append([auc,tp,tn,fp,fn])
    arr = np.array(vals)
    arr_df = \
    pd.DataFrame(
        np.mean(arr,0),
        index=['AUROC','TP','TN','FP','FN'],
        columns=[set_]
    ).T
    arr_dfs.append(
        arr_df.eval(
            '''
            sensitivity = TP/(TP+FN)
            PPV = TP/(TP+FP)
            specificity = TN/(TN+FP)
            NPV = TN/(TN+FN)
            '''
        )
    )

perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance_pgd.csv'
)
perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in perf_df.set_features]
(
    pd.
    concat(arr_dfs).
    join(
        perf_df.
        set_index('set').
        loc[:,['set_features']]
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.
        set_index('Protein')
    ).
    sort_values('AUROC')
)

In [ ]:
perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance.csv'
)
perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in perf_df.set_features]
perf_df = \
(
    perf_df.
    loc[:,['set','set_features']].
    set_index('set').
    join(
        generate_val_scores(
            pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions.csv',
                index_col=0).set_index('set')
        )
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('roc_auc_mean')
)
perf_df

In [ ]:
pgd_perf_df = \
pd.read_csv(
    
    dir_+
    'mortality_predictions_marker_performance_pgd.csv'
)
pgd_perf_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in pgd_perf_df.set_features]
pgd_perf_df = \
(
    pgd_perf_df.
    loc[:,['set','set_features']].
    set_index('set').
    join(
        generate_val_scores(
            pd.read_csv(
                dir_+'mortality_predictions_marker_patient_predictions_pgd.csv',
                index_col=0).set_index('set')
        )
    ).
    reset_index().
    set_index('set_features').
    join(
        idmap_sub.set_index('Protein')
    ).
    sort_values('roc_auc_mean')
)
pgd_fimp_df = pd.read_csv(dir_+'mortality_predictions_marker_feature_importance_pgd.csv')
pgd_fimp_df.set_features = [x.replace("['",'').replace("']",'').strip(", ") for x in pgd_fimp_df.set_features]

pgd_perf_df.to_csv(dir_+'mortality_predictions_marker_processed_patient_predictions_pgd.csv')

In [ ]:
pgd_perf_df.loc[['P00734','P00740','P08697','Q96IY4','P01042-2','D6RAR4']]

In [ ]:
dat = \
(
    pgd_perf_df.
    loc[:,['roc_auc_mean','roc_auc_lwr','roc_auc_upr','set','Gene_name']].
    rename(
        columns={
            'roc_auc_mean' : 'perf',
            'roc_auc_lwr' : 'perf_lwr',
            'roc_auc_upr' : 'perf_upr'
        }
    ).
    reset_index().
    set_index('set').
    join(
        pgd_fimp_df.
        query('Feature!="Intercept"').
        set_index('set').
        loc[:,['mean','2.5%','97.5%']].
        rename(columns={'mean' : 'fimp','2.5%' : 'fimp_lwr','97.5%' : 'fimp_upr'})
    ).
    set_index('set_features')
)

dat = dat.query('set_features!="expired"')
dat = dat.query('set_features!="Survival"')

dat.to_csv(dir_+'mortality_pgd_marker_predictions.csv')


In [ ]:
dat = \
(perf_df.
 reset_index().
 set_index('set_features').
 rename(columns = {'roc_auc_mean' : 'mean'}).
 loc[:,['mean']].
 query('set_features!="PGD" & set_features!="expired"').
 join(
     pgd_perf_df.
     reset_index().
     set_index('set_features').
     rename(columns={'roc_auc_mean' : 'pgd_mean'}).
     loc[:,['pgd_mean']]
 )
)


fig,ax = plt.subplots(dpi=300)
display(dat.sort_values('mean'))
display(dat.sort_values('pgd_mean'))
display(dat.query('mean>0.5 & pgd_mean>0.5').sort_values('pgd_mean'))
ax = sns.scatterplot('mean','pgd_mean',data=dat,ax=ax)
ax.set_xlabel('Survival prediction')
ax.set_ylabel('PGD prediction')

fig.savefig('../../docs/imgs/survival_vs_pgd_marker_prediction.png')

In [ ]:
a=dat['mean'].values
b=dat['pgd_mean'].values
spearmanr(a,b)

In [ ]:
full_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_feature_importance_pgd.csv',
    index_col=0
)
full_perm_fimp_df = \
pd.read_csv(
    dir_+'mortality_predictions_marker_full_permuted_feature_importance_pgd.csv',
    index_col=0
)
proteins = np.intersect1d(full_fimp_df.Feature.unique(),full_perm_fimp_df.Feature.unique())
sig_lst = []
for prot in proteins:
    x = full_fimp_df[full_fimp_df.Feature==prot].Importance.values
    y = full_perm_fimp_df[full_fimp_df.Feature==prot].Importance.values
    lwr, upr = (
        full_perm_fimp_df[
            full_fimp_df.Feature==prot
        ].
        Importance.
        describe(
            percentiles=[0.025,0.975]
        ).
        loc[['2.5%','97.5%']]
    )
    stat, pv = ks_2samp(x,y)
    sig_lst.append(
        pd.DataFrame(
            {'statistic' : stat,'pvalue' : pv,'perm_fimp_lwr' : lwr,'perm_fimp_upr' : upr},
            index=[prot]
        )
    )
sig_df = pd.concat(sig_lst)
sig_df['padj'] = multipletests(sig_df['pvalue'],method='bonferroni')[1]
sig_df

In [ ]:
dat = \
(
    pgd_perf_df.
    loc[:,['roc_auc_mean','roc_auc_lwr','roc_auc_upr','set','Gene_name']].
    rename(
        columns={
            'roc_auc_mean' : 'perf',
            'roc_auc_lwr' : 'perf_lwr',
            'roc_auc_upr' : 'perf_upr'
        }
    ).
    reset_index().
    set_index('set').
    join(
        pgd_fimp_df.
        query('Feature!="Intercept"').
        set_index('set').
        loc[:,['mean','2.5%','97.5%']].
        rename(columns={'mean' : 'fimp','2.5%' : 'fimp_lwr','97.5%' : 'fimp_upr'})
    ).
    set_index('set_features').
    join(sig_df)
)
dat

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perf_lwr>0.5 & perm_fimp_lwr<0 & perm_fimp_upr>0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    ).
    loc[:,
        ['perf_lwr',
         'perf','perf_upr','fimp_lwr',
         'fimp','fimp_upr','padj'
        ]
       ].
    join(
        idmap_sub.set_index('Protein')
    )
)

In [ ]:
(
    dat.
    query(
        'perf>0.5 & perf_lwr>0.5 & perm_fimp_lwr<0 & perm_fimp_upr>0 & (fimp_lwr>0 | fimp_upr<0) & padj<0.0001'
    ).
    loc[:,
        ['perf_lwr',
         'perf','perf_upr','fimp_lwr',
         'fimp','fimp_upr','padj'
        ]
       ].
    join(
        idmap_sub.set_index('Protein')
    )
).to_csv(dir_+'mortality_pgd_significant_prediction_stats.csv')

In [ ]:
Y = joined[['PGD']][['PGD']].copy()

In [ ]:
ppred_df = \
(
    pd.read_csv(
        dir_+
        'mortality_predictions_marker_patient_predictions_pgd.csv',
        index_col=0).
    set_index('set').
    join(
        fimp_df[['set','Feature']].
        query('Feature!="Intercept"').
        drop_duplicates().
        set_index('set')
    ).
    reset_index().
    set_index('Feature').
    join(idmap_sub.set_index('Protein'))
)
tmp = ppred_df[['Sample','y_proba']].set_index('Sample').join(Y)
display(tmp.groupby('PGD')['y_proba'].mean())
fig,ax = plt.subplots(dpi=300)
sns.kdeplot(data=tmp,x='y_proba',hue='PGD',ax=ax)
ax.set_xlim(0.4,0.6)

## GSEA

In [ ]:
import gseapy as gp
gp.__version__

In [ ]:
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
col_map = { 'nes' : 'Normalized Enrichment Score', 'pval' : 'P-value', 'fdr' : 'False Discovery Rate',"Category" : 'Category'}

### Survival

In [ ]:
cov = ''
rnk = \
pd.read_csv(dir_+'survival_rank_statistic.csv')[['Gene_name','odds']].drop_duplicates(subset='Gene_name')
rnk.head()

In [ ]:
genes = ['F2','F9','SERPINF2','CPB2','SERPIND1','CLU','F5','MASP1','KLKB1','F10','PLG','PROS1',
         'C6','SERPINA1','C1QA','CFD','C1QB','SERPINC1','VWF','F12','C1R',
         'C7','C9','SERPINA5','C2','CFI','MASP2','C1S','C8G','CFH','C1QC',
         'C3','F13B','FGB','KNG1']
ledge_genes = ['F2','F9','SERPINF2','CPB2','SERPIND1','CLU']
display(rnk.query('Gene_name in @genes').sort_values('Gene_name'))
display(rnk.query('Gene_name in @ledge_genes'))

In [ ]:
for g in gs:
    print('\t'+g)
    pre_res = gp.prerank(rnk=rnk[['Gene_name','odds']], gene_sets=g,
                     processes=4,
                     permutation_num=10000, 
                     outdir=dir_+'survival_gsea/'+g,format='png')

In [ ]:
datas=[]
for g in gs:
    data = (
        pd.read_csv(
            dir_+'survival_gsea/'+g+'/gseapy.prerank.gene_sets.report.csv'
        ).sort_values(['fdr','nes'],ascending=[True,False]).
        query('fdr < 0.2 & (nes > 0 | nes < 0) & (fdr>pval | fdr==0)').
        rename(columns=col_map).
        set_index('Term')
    )
    data['Category'] = g
    datas.append(data)
pd.concat(datas).to_csv(dir_+'survival_gsea/'+
                       'prerank_report_all_categories.csv')
pd.concat(datas).shape

In [ ]:
enriched = pd.concat(datas).copy()
tmp = enriched[[k for k in col_map.values()]].sort_values('False Discovery Rate',ascending=True).round(4)
tmp.sort_values('False Discovery Rate',ascending=True).to_csv(dir_+'survival_gsea/'+'pathways_functions.csv')
enriched.to_csv(dir_+'survival_gsea/'+'pathways_functions_wgenes.csv')
display(tmp.sort_values('Normalized Enrichment Score'))
print(tmp.shape[0])

In [ ]:
tmp['lognes'] = np.log(tmp['Normalized Enrichment Score'])
tmp.sort_values('Normalized Enrichment Score')['lognes'].plot()

### STRING

In [ ]:
import glob
lst = glob.glob(dir_+'enrichment*.tsv')
(
    pd.concat([pd.read_table(x,sep='\t') for x in lst]).
    sort_values('false discovery rate').
    to_csv(dir_+'tah_string_enrichment.csv')
)
(
    pd.concat([pd.read_table(x,sep='\t') for x in lst]).
    sort_values('false discovery rate')
)

In [ ]:
import glob
lst = glob.glob('/Users/nickgiangreco/Downloads/enrichment*.tsv')
(
    pd.concat([pd.read_table(x,sep='\t') for x in lst]).
    sort_values('false discovery rate').
    to_csv(dir_+'mortality_lvad_pgd_string_enrichment.csv')
)
(
    pd.concat([pd.read_table(x,sep='\t') for x in lst]).
    sort_values('false discovery rate')
)